<a href="https://colab.research.google.com/github/AVJdataminer/AVJdataminer.github.io/blob/master/Predict_consumption_with_predicted_temperatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the modules and list files in my directory.

In [2]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import datetime
import os
os.listdir()

['sample_data',
 '.config',
 'pred_submit20181017-2209.csv',
 'pred_submit20181017-2206.csv',
 'temperatures_to_train_on_20181017-1116.csv',
 'temperatures_pred_submit20181017-1912.csv',
 'data_to_train_on_20181017-1359.csv',
 'temperatures_to_train_on_20181017-1022.csv',
 'temperatures_pred_trntest20181017-1919.csv']

Load data 

In [3]:
url="data_to_train_on_20181017-1359.csv"
train=pd.read_csv(url)
train.drop(['Unnamed: 0','timestamp'], axis=1, inplace=True)
train.head()

,series_id,consumption,temperature,wkday,hour,day_of_yr,year
0,103088,143709.717612,-4.675,1,17,364,2014
1,103088,157424.927949,-5.000,1,18,364,2014
2,103088,153370.363034,-4.400,2,14,365,2014
3,103088,137602.089769,-4.650,2,15,365,2014
4,103088,131771.016064,-5.100,2,16,365,2014


In [1]:
#scale input data
y=train['consumption'].values
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(train.drop(['consumption'], axis=1).values)
#split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

NameError: ignored

In [0]:
y_train

In [0]:
from sklearn.preprocessing import MinMaxScaler
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

In [0]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 100

# Store training stats
history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])

In [0]:
[loss, mae] = model.evaluate(X_test, y_test, verbose=0)
print("Testing set Mean Abs Error: {:7.2f}".format(mae * 1))

In [0]:
test_pred = model.predict(X_test).flatten()
plt.scatter(y_test, test_pred)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
_ = plt.plot([-10, 10], [-10, 10])

In [0]:
test_pred

In [0]:
url="https://s3.amazonaws.com/drivendata/data/55/public/submission_format.csv"
submit=pd.read_csv(url)
#submit.dropna(inplace=True)
#get predicted temps to fill na's
df=pd.read_csv('temperatures_pred_submit20181017-1912.csv')
submit['temperature']=submit['temperature'].fillna(df['temp_pred'])
submit['temperature'].fillna(0, inplace=True)
submit.isnull().sum()


In [0]:
submit.set_index('timestamp')
submit['timestamp'] =  pd.to_datetime(submit['timestamp'], format="%Y-%m-%d %H:%M:%S")
submit['wkday']=submit['timestamp'].dt.weekday
submit['hour']=submit['timestamp'].dt.strftime('%H').astype(int)
submit['day_of_yr']=submit['timestamp'].dt.strftime('%j').astype(int)
submit["year"] = submit["timestamp"].dt.year
submit.drop(['pred_id','consumption','timestamp','prediction_window'], axis=1, inplace=True)
submit.head()
X=sc.fit_transform(submit)
X

In [0]:
url="https://s3.amazonaws.com/drivendata/data/55/public/submission_format.csv"
submit_out=pd.read_csv(url)
#submit_out.dropna(inplace=True)
submit_out['consumption'] = model.predict(X).flatten()
submit_out.head()
submit_out.set_index('pred_id')

In [0]:
#build the predictions csv
file_time = 'pred_submit' + datetime.datetime.now().strftime("%Y%m%d-%H%M")+'.csv'
submit_out.to_csv(file_time)

In [0]:
len(submit_out)

In [0]:
[loss, mae] = model.evaluate(X, submit_out['consumption'], verbose=0)
print("Testing set Mean Abs Error: {:7.2f}".format(mae * 1))

In [0]:
error = (submit_out['cons_pred']) - submit_out['consumption']
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")